In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,Wav2Vec2Processor
from transformers import WhisperForConditionalGeneration,Wav2Vec2ForCTC

from transformers.generation.configuration_utils import GenerationConfig
from transformers import T5ForConditionalGeneration
from functions import *
import torch
import torch.nn as nn
from transformers.modeling_outputs import BaseModelOutput

Merge two models and save

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5")
# use whisper as encoder
whisper_model = WhisperForConditionalGeneration.from_pretrained('/home/zhenlan/Desktop/Projects/Bengali ASR/model_all3')
model.encoder = whisperEncoderWhead(whisper_model.get_encoder(),768,768)
torch.save(model, '/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model_whisper/t5.pth')
# use wav2vec
# wav2vec = Wav2Vec2ForCTC.from_pretrained("Sameen53/training_45k")
# model.encoder = whisperEncoderWhead(wav2vec.wav2vec2,1024,768)
# torch.save(model, '/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model_wav2vec/t5.pth')

Load merged model

In [ ]:
import torch
from transformers.modeling_outputs import BaseModelOutput

In [ ]:
class whisperEncoderWhead(torch.nn.Module):
    def __init__(self, whisper) -> None:
        super().__init__()
        self.convert = torch.nn.Linear(768,768)
        self.whisper = whisper

    def forward(self,speech,*args,**kwargs):
        out = self.whisper(speech)[0]
        out = self.convert(out)
        return BaseModelOutput(out)

In [3]:
model = torch.load('/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model/t5.pth')

In [24]:
for param in model.encoder.whisper.parameters():
    param.requires_grad = False
for param in model.decoder.parameters():
    param.requires_grad = False

In [37]:
speech_input = torch.rand(4, 80, 16)
txt_input = torch.randint(0,128,(4,16))
attention_mask = torch.rand((4,16))>0.3
# out = model(input_ids=speech_input,labels=txt_input)
logits = model(input_ids=speech_input,decoder_input_ids=txt_input).logits
loss = torch.sum(torch.nn.CrossEntropyLoss(reduction='none')(logits[:,0:-1].reshape(-1,32128),\
                                                              txt_input[:,1:].reshape(-1))*attention_mask[:,1:].reshape(-1))/torch.sum(attention_mask[:,1:])

Generation

In [18]:
import json
with open('/home/zhenlan/Desktop/Projects/Bengali ASR/T5Model/generation_config.json', 'r') as file:
    generation_config = json.load(file)

In [19]:
generation_config = GenerationConfig.from_dict(generation_config)

In [35]:
encoder_outputs = model.encoder(speech_input)
out = model.generate(encoder_outputs=encoder_outputs,generation_config=generation_config)

In [32]:
len(encoder_outputs)

1

In [28]:
encoder_outputs[0].shape

torch.Size([4, 8, 768])